# Data Engineering Capstone Project - Metro_Madrid

## Project Summary
This notebook gets output data from [metro-big-data-unir](https://github.com/juananthony/metro-big-data-unir) project and create a database to analysis.

[*Metro de Madrid*](https://www.metromadrid.es/) is the name of the tube/subway service that operates in Madrid, Spain. This service has 302 stations on 13 lines plus a light rail system called *Metro Ligero*. This service is used, on average in 2020, more than 30 million times each month.

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [3]:
import os
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('udacity-capstone').getOrCreate()

## Step 1: Scope the Project and Gather Data

### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

In [5]:
DATA_PATH = './data'
lines_file = 'lines.csv'
stations_file = 'stations.csv'
mentions_file = 'mentions_20210126.csv'


In [8]:
lines = spark.read.csv(os.path.join(DATA_PATH, lines_file), header=True)
lines.show()

+----+
|line|
+----+
|  L9|
| ML3|
|  L8|
| ML1|
| ML2|
|  L7|
| L11|
|  L5|
|  L6|
| L10|
|  L3|
|  L4|
|  L2|
| L12|
|  L1|
|   R|
+----+



In [11]:
stations = spark.read.csv(os.path.join(DATA_PATH, stations_file), header=True)
stations.show(5)

+----------------+----+
|         station|line|
+----------------+----+
|   Paco de Lucía|  L9|
|      Mirasierra|  L9|
|    Herrera Oria|  L9|
|Barrio del Pilar|  L9|
|        Ventilla|  L9|
+----------------+----+
only showing top 5 rows



In [14]:
mentions = spark.read.csv(os.path.join(DATA_PATH, mentions_file), header=True, multiLine=True, escape='"')
mentions.show(1)

+--------------------+--------------------+--------------------+-------------------+---------------------+------------------+-----------------+----------------+-----------+-----------------+-----------------------+------------------------+---------------------------------+
|                 _id|          created_at|                text|                 id|in_reply_to_status_id|           user.id|        user.name|user.screen_name|description|profile_image_url|profile_image_url_https|extended_tweet.full_text|classification.naive_bayes.result|
+--------------------+--------------------+--------------------+-------------------+---------------------+------------------+-----------------+----------------+-----------+-----------------+-----------------------+------------------------+---------------------------------+
|ObjectId(5be9f6d0...|Mon Nov 12 21:55:...|@metro_madrid Qué...|1062101577610215424|  1060150880522121216|702578868503388160|María Agustí Sanz|     magustisanz|       null|      

In [16]:
mentions.groupBy('`classification.naive_bayes.result`').count().show()

+---------------------------------+------+
|classification.naive_bayes.result| count|
+---------------------------------+------+
|                             null| 83101|
|                          nothing|693429|
|                        complaint|111098|
|                            issue| 59301|
+---------------------------------+------+



## Step 2: Explore and Assess the Data
### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

### Cleaning Steps
Document steps necessary to clean the data

In [20]:
from pyspark.sql.functions import col
mentions = mentions.filter(col('`classification.naive_bayes.result`').isin(['complaint', 'issue']))

## Step 3: Define the Data Model
### 3.1 Conceptual Data Model
The data we want to store is all messages that inform about any issue or complaint in a line or a station even if one message inform about an issue that affect two different lines. That the reason why the fact table is the inform fact, that can be a complaint or an issue. One tweet can inform about an issue that affect two lines (i.e.: a closed station and all lines that stops there). In other words, one tweet generates one or many "inform fact" records.
![fact-dimension diagram](./img/class_diagram.png "Fact-Dimension Diagram")

### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

## Step 4: Run Pipelines to Model the Data 
### 4.1 Create the data model
Build the data pipelines to create the data model.

### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.